In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import requests
import numpy as np
import pandas as pd

In [2]:
# markets = requests.get("https://api.bittrex.com/api/v1.1/public/getmarkets")
# markets_json = markets.json()["result"]
# markets_json[:2]

In [3]:
# markets_df = pd.json_normalize(markets_json)
# # markets_df.iloc[:,[0,1,2,3,4,5,6,7,8]]
# markets_df[["MarketName"]]

In [4]:
# exchanges_df = pd.DataFrame()

# for market_name in list(markets_df[["MarketName"]].values[:4]):
# #     print(market_name[0])
#     exchange = requests.get(f"https://api.bittrex.com/api/v1.1/public/getmarkethistory?market={market_name[0]}")
# #     exchange_df = pd.json_normalize(exchange.json()["result"][:3])
#     exchange_df = pd.json_normalize(exchange.json()["result"])
#     exchanges_df[[f"{market_name}-timestamp"]] = exchange_df[["TimeStamp"]]
#     exchanges_df[[f"{market_name}-price"]] = exchange_df[["Price"]]
    
# exchanges_df

### Pobranie z API

In [15]:
market_history_df_dict = {}
markets = ["BTC-LTC", "BTC-DOGE", "BTC-VTC"]

for market_name in markets:
    market_history = requests.get(f"https://api.bittrex.com/api/v1.1/public/getmarkethistory?market={market_name}")
    market_history_json = market_history.json()["result"]
    market_history_df = pd.json_normalize(market_history_json)
    market_history_df = market_history_df[["TimeStamp", "Quantity", "Price", "Total", "OrderType"]]
    market_history_df_dict[market_name] = market_history_df
    
market_history_df_dict

{'BTC-LTC':                  TimeStamp   Quantity     Price     Total OrderType
 0   2021-05-30T12:34:30.16   6.965028  0.004915  0.034232      SELL
 1   2021-05-30T12:33:52.83   2.512791  0.004916  0.012353       BUY
 2   2021-05-30T12:28:16.44   5.647740  0.004923  0.027801      SELL
 3   2021-05-30T12:26:34.76   0.038861  0.004927  0.000191       BUY
 4   2021-05-30T12:25:27.89   0.640084  0.004920  0.003149      SELL
 ..                     ...        ...       ...       ...       ...
 95   2021-05-30T09:43:07.4   3.786570  0.004925  0.018649      SELL
 96  2021-05-30T09:42:47.68  10.887593  0.004922  0.053589      SELL
 97  2021-05-30T09:42:47.68  22.639143  0.004923  0.111453      SELL
 98  2021-05-30T09:42:41.05   0.000308  0.004921  0.000002       BUY
 99  2021-05-30T09:42:40.89  22.640000  0.004921  0.111408       BUY
 
 [100 rows x 5 columns],
 'BTC-DOGE':                  TimeStamp     Quantity     Price     Total OrderType
 0   2021-05-30T12:44:32.35  1000.000000  0.000009 

In [6]:
# market_history = requests.get(f"https://api.bittrex.com/api/v1.1/public/getmarkethistory?market=BTC-LTC")
# market_history_json = market_history.json()["result"]
# market_history_df = pd.json_normalize(market_history_json)
# market_history_df.loc[1]

In [7]:
# markets = requests.get("https://api.bittrex.com/api/v1.1/public/getmarkets")
# markets_json = markets.json()["result"]
# markets_df = pd.json_normalize(markets_json)
# num_of_markets = len(markets_df["MarketName"])

## Stworzenie tabel

In [8]:
transakcja_df = pd.DataFrame(
    columns=["IdCzasu",
             "IdMarketu",
             "IdRodzajuTransakcji",
             "Ilość",
             "Cena",
             "Suma"])

czas_df = pd.DataFrame(
    columns=["IdCzasu",
             "Data",
             "Dzień",
             "Miesiąc",
             "Rok"])

market_df = pd.DataFrame(
    columns=["IdMarketu",
             "Nazwa"])

rodzaj_transakcji_df = pd.DataFrame(
    columns=["IdRodzajuTransakcji",
             "NazwaRodzajuTransakcji"])

### Transakcja

In [16]:
transakcja_df.drop(transakcja_df.index, inplace=True)
market_name_id = 0

for market_name in markets:
    time_stamps = market_history_df_dict[market_name]["TimeStamp"]
    id_rodzaju_transakcji = market_history_df_dict[market_name]["OrderType"]
    id_rodzaju_transakcji = id_rodzaju_transakcji.replace({"BUY": 0, "SELL": 1})
    quantity = market_history_df_dict[market_name]["Quantity"]
    price = market_history_df_dict[market_name]["Price"]
    total = market_history_df_dict[market_name]["Total"]

    temp_df = pd.DataFrame(
        {"IdCzasu": time_stamps,
         "IdMarketu": market_name_id,
         "IdRodzajuTransakcji": id_rodzaju_transakcji,
         "Ilość": quantity,
         "Cena": price,
         "Suma": total})
    market_name_id = market_name_id + 1
    transakcja_df = pd.concat([transakcja_df, temp_df])
    
transakcja_df = transakcja_df.reset_index(drop=True)
transakcja_df

,IdCzasu,IdMarketu,IdRodzajuTransakcji,Ilość,Cena,Suma
0,2021-05-30T12:34:30.16,0,1,6.965028,0.004915,0.034232
1,2021-05-30T12:33:52.83,0,0,2.512791,0.004916,0.012353
2,2021-05-30T12:28:16.44,0,1,5.647740,0.004923,0.027801
3,2021-05-30T12:26:34.76,0,0,0.038861,0.004927,0.000191
4,2021-05-30T12:25:27.89,0,1,0.640084,0.004920,0.003149
...,...,...,...,...,...,...
295,2021-05-30T09:29:18.4,2,1,58.183933,0.000019,0.001121
296,2021-05-30T09:25:45.39,2,1,182.773783,0.000019,0.003535
297,2021-05-30T09:25:45.39,2,1,103.359173,0.000019,0.002000
298,2021-05-30T09:17:25.23,2,1,8.042365,0.000019,0.000155


### Czas

In [10]:
czas_df.drop(czas_df.index, inplace=True)
czas_df["IdCzasu"] = transakcja_df["IdCzasu"]
id_czasu_datetime = pd.to_datetime(czas_df["IdCzasu"],
                                   format="%Y-%m-%dT%H:%M:%S.%f")
days = [x.day for x in id_czasu_datetime]
months = [x.month for x in id_czasu_datetime]
years = [x.year for x in id_czasu_datetime]

czas_df["Data"] = [f"{x.year}-{x.month}-{x.day}" for x in id_czasu_datetime]
czas_df["Dzień"] = days
czas_df["Miesiąc"] = months
czas_df["Rok"] = years

czas_df

# czas_df["Data"] = id_czasu_datetime
# czas_df["Dzień"] = 

,IdCzasu,Data,Dzień,Miesiąc,Rok
0,2021-05-30T12:34:30.16,2021-5-30,30,5,2021
1,2021-05-30T12:33:52.83,2021-5-30,30,5,2021
2,2021-05-30T12:28:16.44,2021-5-30,30,5,2021
3,2021-05-30T12:26:34.76,2021-5-30,30,5,2021
4,2021-05-30T12:25:27.89,2021-5-30,30,5,2021
...,...,...,...,...,...
295,2021-05-30T09:03:14.94,2021-5-30,30,5,2021
296,2021-05-30T09:03:14.94,2021-5-30,30,5,2021
297,2021-05-30T09:03:14.94,2021-5-30,30,5,2021
298,2021-05-30T09:03:14.94,2021-5-30,30,5,2021


### Market

In [11]:
market_df.drop(market_df.index, inplace=True)
market_df["IdMarketu"] = list(range(len(markets)))
market_df["Nazwa"] = markets
# market_df["IdMarketu"] = list(range(num_of_markets))
# market_df["Nazwa"] = markets_df["MarketName"]
market_df.head()

,IdMarketu,Nazwa
0,0,BTC-LTC
1,1,BTC-DOGE
2,2,BTC-VTC


### RodzajTransakcji

In [12]:
rodzaj_transakcji_df.drop(rodzaj_transakcji_df.index, inplace=True)
rodzaj_transakcji_df.loc[0] = [0, "Buy"]
rodzaj_transakcji_df.loc[1] = [1, "Sell"]
rodzaj_transakcji_df

,IdRodzajuTransakcji,NazwaRodzajuTransakcji
0,0,Buy
1,1,Sell


## Wszystkie tabele

In [17]:
transakcja_df
czas_df
market_df
rodzaj_transakcji_df

,IdCzasu,IdMarketu,IdRodzajuTransakcji,Ilość,Cena,Suma
0,2021-05-30T12:34:30.16,0,1,6.965028,0.004915,0.034232
1,2021-05-30T12:33:52.83,0,0,2.512791,0.004916,0.012353
2,2021-05-30T12:28:16.44,0,1,5.647740,0.004923,0.027801
3,2021-05-30T12:26:34.76,0,0,0.038861,0.004927,0.000191
4,2021-05-30T12:25:27.89,0,1,0.640084,0.004920,0.003149
...,...,...,...,...,...,...
295,2021-05-30T09:29:18.4,2,1,58.183933,0.000019,0.001121
296,2021-05-30T09:25:45.39,2,1,182.773783,0.000019,0.003535
297,2021-05-30T09:25:45.39,2,1,103.359173,0.000019,0.002000
298,2021-05-30T09:17:25.23,2,1,8.042365,0.000019,0.000155


,IdCzasu,Data,Dzień,Miesiąc,Rok
0,2021-05-30T12:34:30.16,2021-5-30,30,5,2021
1,2021-05-30T12:33:52.83,2021-5-30,30,5,2021
2,2021-05-30T12:28:16.44,2021-5-30,30,5,2021
3,2021-05-30T12:26:34.76,2021-5-30,30,5,2021
4,2021-05-30T12:25:27.89,2021-5-30,30,5,2021
...,...,...,...,...,...
295,2021-05-30T09:03:14.94,2021-5-30,30,5,2021
296,2021-05-30T09:03:14.94,2021-5-30,30,5,2021
297,2021-05-30T09:03:14.94,2021-5-30,30,5,2021
298,2021-05-30T09:03:14.94,2021-5-30,30,5,2021


,IdMarketu,Nazwa
0,0,BTC-LTC
1,1,BTC-DOGE
2,2,BTC-VTC


,IdRodzajuTransakcji,NazwaRodzajuTransakcji
0,0,Buy
1,1,Sell
